# 文本预处理
⽂本的常⻅预处理步骤，这些步骤通常包括：
+ 1. 将⽂本作为字符串加载到内存中。
+ 2. 将字符串拆分为词元（如单词和字符）。
+ 3. 建⽴⼀个词表，将拆分的词元映射到数字索引。
+ 4. 将⽂本转换为数字索引序列，⽅便模型操作。


In [1]:
import collections
import re
from d2l import torch as d2l

## 1. 读取数据集
⾸先，我们从H.G.Well的时光机器99中加载⽂本。这是⼀个相当⼩的语料库，只有30000多个单词，但⾜够我们⼩试⽜⼑，⽽现实中的⽂档集合可能会包含数⼗亿个单词。下⾯的函数将数据集读取到由多条⽂本⾏组成的列表中，其中每条⽂本⾏都是⼀个字符串。为简单起⻅，我们在这⾥忽略了标点符号和字⺟⼤写。


In [2]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                    '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine(): #@save
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## 2. 词元化
下⾯的tokenize函数将⽂本⾏列表（lines）作为输⼊，列表中的每个元素是⼀个⽂本序列（如⼀条⽂本⾏）。
每个⽂本序列⼜被拆分成⼀个词元列表，词元（token）是⽂本的基本单位。最后，返回⼀个由词元列表组成
的列表，其中的每个词元都是⼀个字符串（string）。

In [3]:
def tokenize(lines, token='word'): #@save
    # 将文本划分为一个一个独立的词列表
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char': 
        return [list(line) for line in lines]
    else:
        print('Error: undefined token type: ' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## 3， 词表
词元的类型是字符串，⽽模型需要的输⼊是数字，因此这种类型不⽅便模型使⽤。现在，让我们构建⼀个字
典，通常也叫做词表（vocabulary），⽤来将字符串类型的词元映射到从0开始的数字索引中。我们先将训练
集中的所有⽂档合并在⼀起，对它们的唯⼀词元进⾏统计，得到的统计结果称之为语料（corpus）。然后根
据每个唯⼀词元的出现频率，为其分配⼀个数字索引。很少出现的词元通常被移除，这可以降低复杂性。另
外，语料库中不存在或已删除的任何词元都将映射到⼀个特定的未知词元“unk”。我们可以选择增加⼀个
列表，⽤于保存那些被保留的词元，例如：填充词元（“pad”）；序列开始词元（“bos”）；序列结束词元
（“eos”）。


In [4]:
class Vocab: #@save
    # 文本词表
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按照单词的出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                  reverse=True)
        # 未知词元的索引为0
        # 索引对应的词元
        self.idx_to_token = ['<unk>'] + reserved_tokens
        # 词元对应的索引
        self.token_to_idx = {
            token: idx
            for idx, token in enumerate(self.idx_to_token)
        }
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
      
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx[tokens]
        return [self.__getitem__(token) for token in tokens]
    
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    
    @property
    def unk(self): # 未知词元的索引为0
        return 0
    
    @property
    def token_freqs(self):
        return self._token_freqs
    
    
def count_corpus(tokens): #@save
    # 统计词元的频率
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [5]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [6]:
for i in [0, 10]:
    print('文本: ', tokens[i])
    print('索引: ', vocab[tokens[i]])

文本:  ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引:  [1, 19, 50, 40, 2183, 2184, 400]
文本:  ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引:  [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


## 4. 整合所有功能
在使⽤上述函数时，我们将所有功能打包到load_corpus_time_machine函数中，该函数返回corpus（词元索
引列表）和vocab（时光机器语料库的词表）。

In [7]:
def load_corpus_time_machine(max_tokens=-1): #@save
    # 返回数据集词元索引表和词表
    lines = read_time_machine()
    tokens = tokenize(lines)
    vocab = Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[: max_tokens] # 词元索引限制长度
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(32775, 4580)

## 总结
+ ⽂本是序列数据的⼀种最常⻅的形式之⼀。
+ 为了对⽂本进⾏预处理，我们通常将⽂本拆分为词元，构建词表将词元字符串映射为数字索引，并将⽂本数据转换为词元索引以供模型操作。